<a href="https://colab.research.google.com/github/SaenkoDenis/machine-learning-py/blob/main/data-analysis/pandas-datetime/rabota_c_datetime_b_pandas_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Этот нотбук для статьи в блоге [Работа с datetime в Pandas DataFrame](http://i922161r.beget.tech/2021/05/23/rabota-c-datetime-b-pandas-dataframe/)

In [ ]:
import pandas as pd
import numpy as np

## 1. Преобразование строк в datetime

In [ ]:
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'])
df

,date,value
0,2000-03-10,2
1,2000-03-11,3
2,2000-03-12,4


In [3]:
df = pd.DataFrame({'date': ['3/10/2000', '3/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df

,date,value
0,2000-10-03,2
1,2000-11-03,3
2,2000-12-03,4


In [4]:
df = pd.DataFrame({'date': ['2016-6-10 20:30:0', 
                            '2016-7-1 19:45:30', 
                            '2013-10-12 4:5:1'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'], format="%Y-%d-%m %H:%M:%S")
df

,date,value
0,2016-10-06 20:30:00,2
1,2016-01-07 19:45:30,3
2,2013-12-10 04:05:01,4


In [5]:
# Make up 3000 rows
df = pd.DataFrame({'date': ['3/11/2000', '3/12/2000', '3/13/2000'] * 1000 })

%timeit pd.to_datetime(df['date'], infer_datetime_format=True)
# >> 100 loops, best of 3: 10.4 ms per loop

%timeit pd.to_datetime(df['date'], infer_datetime_format=False)
# >> 1 loop, best of 3: 471 ms per loop

1000 loops, best of 5: 1.52 ms per loop
1000 loops, best of 5: 1.49 ms per loop


In [6]:
df = pd.DataFrame({'date': ['3/10/2000', 'a/11/2000', '3/12/2000'],
                   'value': [2, 3, 4]})
df['date'] = pd.to_datetime(df['date'])
df

ParserError: ignored

In [7]:
df['date'] = pd.to_datetime(df['date'], errors='ignore')
df

,date,value
0,3/10/2000,2
1,a/11/2000,3
2,3/12/2000,4


In [8]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df

,date,value
0,2000-03-10,2
1,NaT,3
2,2000-03-12,4


## 2. Соберите дату и время из нескольких столбцов

In [9]:
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5]})

df['date'] = pd.to_datetime(df)
df

,year,month,day,date
0,2015,2,4,2015-02-04
1,2016,3,5,2016-03-05


## 3. Получить год, месяц и день

In [10]:
df = pd.DataFrame({'name': ['Tom', 'Andy', 'Lucas'],
                 'DoB': ['08-05-1997', '04-28-1996', '12-16-1995']})
df['DoB'] = pd.to_datetime(df['DoB'])
df

,name,DoB
0,Tom,1997-08-05
1,Andy,1996-04-28
2,Lucas,1995-12-16


In [11]:
df['year']= df['DoB'].dt.year
df['month']= df['DoB'].dt.month
df['day']= df['DoB'].dt.day
df

,name,DoB,year,month,day
0,Tom,1997-08-05,1997,8,5
1,Andy,1996-04-28,1996,4,28
2,Lucas,1995-12-16,1995,12,16


## 4. Получите неделю года, день недели и високосный год


In [12]:
df['week_of_year'] = df['DoB'].dt.week
df['day_of_week'] = df['DoB'].dt.dayofweek
df['is_leap_year'] = df['DoB'].dt.is_leap_year
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year
0,Tom,1997-08-05,1997,8,5,32,1,False
1,Andy,1996-04-28,1996,4,28,17,6,True
2,Lucas,1995-12-16,1995,12,16,50,5,False


In [13]:
dw_mapping={
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday',
    5: 'Saturday', 
    6: 'Sunday'
} 
df['day_of_week_name']=df['DoB'].dt.weekday.map(dw_mapping)
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday


## 5. Получить возраст с даты рождения

In [14]:
today = pd.to_datetime('today')
df['age'] = today.year - df['DoB'].dt.year
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,24
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,25
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,26


In [15]:
# Year difference
today = pd.to_datetime('today')
diff_y = today.year - df['DoB'].dt.year
# Haven't had birthday
b_md = df['DoB'].apply(lambda x: (x.month,x.day) )
no_birthday = b_md > (today.month,today.day)

df['age'] = diff_y - no_birthday
df

,name,DoB,year,month,day,week_of_year,day_of_week,is_leap_year,day_of_week_name,age
0,Tom,1997-08-05,1997,8,5,32,1,False,Tuesday,23
1,Andy,1996-04-28,1996,4,28,17,6,True,Sunday,25
2,Lucas,1995-12-16,1995,12,16,50,5,False,Saturday,25


## 6. Повышение производительности за счет установки столбца даты в качестве индекса

In [17]:
# condition = (df['date'] > start_date) & (df['date'] <= end_date)
# df.loc[condition]

In [ ]:
# df = pd.read_csv('data/city_sales.csv',parse_dates=['date'])
# df.info()

# RangeIndex: 1795144 entries, 0 to 1795143
# Data columns (total 3 columns):
#  #   Column  Dtype         
# ---  ------  -----         
#  0   date    datetime64[ns]
#  1   num     int64         
#  2   city    object        
# dtypes: datetime64[ns](1), int64(1), object(1)
# memory usage: 41.1+ MB

In [ ]:
df = df.set_index(['date'])
df

## 7. Выберите данные за определенный год и выполните агрегирование

In [ ]:
df.loc['2018']

In [ ]:
df.loc['2018','num'].sum()
>> 1231190

In [ ]:
df['2018'].groupby('city').sum()

# 8. Выберите данные с определенным месяцем и определенным днем месяца

In [ ]:
df.loc['2018-5']

In [ ]:
df.loc['2018-5-1']

## 9. Выберите данные между двумя датами

In [ ]:
df.loc['2016' : '2018']

In [ ]:
df.loc['2018-5-2 10':'2018-5-2 11']

In [ ]:
df.loc['2018-5-2 10:30':'2018-5-2 10:45']

In [ ]:
df.between_time('10:30','10:45')

## 10. Обработка отсутствующих значений

In [ ]:
df['rolling_sum'] = df.rolling(3).sum()
df.head()

In [ ]:
df['rolling_sum_backfilled'] = df['rolling_sum'].fillna(method='backfill')
df.head()